In [7]:
import pandas as pd
import json
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
print(pd.__version__)
df1 = pd.read_json('radical/analyse_fase-kenmerken_onderzoek/Data/yt_results_uniq_clean.jsonl', lines=True)
df1.info()

df2 = pd.read_json('radical/analyse_fase-kenmerken_onderzoek/Data/yt_results_archive_clean.jsonl', lines=True)

model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

3.0.0
<class 'pandas.DataFrame'>
RangeIndex: 3119 entries, 0 to 3118
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   video_id           3119 non-null   str           
 1   video_url          3119 non-null   str           
 2   scraped_at         3119 non-null   datetime64[ms]
 3   title              3119 non-null   str           
 4   description        3119 non-null   str           
 5   channel_title      3119 non-null   str           
 6   published_at       3119 non-null   datetime64[ms]
 7   views              3119 non-null   int64         
 8   likes              3119 non-null   int64         
 9   comment_count      3119 non-null   int64         
 10  duration_seconds   3119 non-null   int64         
 11  keyword_relevance  3119 non-null   object        
 12  _iteration         3119 non-null   int64         
 13  _archived_at       3119 non-null   datetime64[ms]
dtypes: datetime64

Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

In [8]:
df2.info()

<class 'pandas.DataFrame'>
RangeIndex: 5341 entries, 0 to 5340
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   video_id           5341 non-null   str           
 1   video_url          5341 non-null   str           
 2   scraped_at         5341 non-null   datetime64[ms]
 3   title              5341 non-null   str           
 4   description        5341 non-null   str           
 5   channel_title      5341 non-null   str           
 6   published_at       5341 non-null   datetime64[ms]
 7   views              5341 non-null   int64         
 8   likes              5341 non-null   int64         
 9   comment_count      5341 non-null   int64         
 10  duration_seconds   5341 non-null   int64         
 11  keyword_relevance  5341 non-null   object        
 12  _iteration         5341 non-null   int64         
 13  _archived_at       5341 non-null   datetime64[ms]
dtypes: datetime64[ms](3

In [9]:
def predict_in_batches(texts, batch_size=32):
    all_labels = []
    all_confidences = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        with torch.no_grad():
            outputs = model(**inputs)

        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

        sentiment_map = {
            0: "Very Negative",
            1: "Negative",
            2: "Neutral",
            3: "Positive",
            4: "Very Positive"
        }

        for p in probs:
            idx = torch.argmax(p).item()
            all_labels.append(sentiment_map[idx])
            all_confidences.append(float(p[idx]))

    return all_labels, all_confidences

In [10]:
labels, confs = predict_in_batches(df1["description"].astype(str).tolist(), batch_size=16)

df1["sentiment"] = labels
df1["confidence"] = confs

labels, confs = predict_in_batches(df2["description"].astype(str).tolist(), batch_size=16)

df2["sentiment"] = labels
df2["confidence"] = confs

#exporteren van gelabelde dataset
df1.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/yt-results-uniq-sentiment-labeld.jsonl', orient='records', lines=True)
df2.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/yt-results-archive-sentiment-labeld.jsonl', orient='records', lines=True)

/tmp/ipykernel_69956/1426163866.py:12: Pandas4Warning: The default 'epoch' date format is deprecated and will be removed in a future version, please use 'iso' date format instead.
  df1.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/yt-results-uniq-sentiment-labeld.jsonl', orient='records', lines=True)
/tmp/ipykernel_69956/1426163866.py:13: Pandas4Warning: The default 'epoch' date format is deprecated and will be removed in a future version, please use 'iso' date format instead.
  df2.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/yt-results-archive-sentiment-labeld.jsonl', orient='records', lines=True)
